<a href="https://colab.research.google.com/github/jinquan122/Sales-Data/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dataset
import pandas as pd
file = pd.ExcelFile("/content/da.test.xlsx")

In [2]:
order = file.parse(sheet_name = "Order Data")
payment = file.parse(sheet_name = "Payments Data")

In [3]:
# Display number of obsesrvation
a = len(order.index)
b = len(payment.index)

print("Total observation of order data : ",a)
print("Total observation of payment data : ",b)

Total observation of order data :  7987
Total observation of payment data :  7838


In [4]:
#Check missing value
order.isnull().sum()

orderid            0
shopid             0
userid             0
checkoutid         0
total_price        0
currency           0
status             0
created_time       0
payment_time    1703
cancel_time     5359
dtype: int64

In [5]:
payment.isnull().sum()

checkoutid           0
total_price          0
currency             0
payment_method       0
payment_time      1662
dtype: int64

In [6]:
#Paid = 2, 11, 12, 13
#Unpaid and Cancelled = 6
#Paid then Cancelled = 7, 8, 9, 10
order["status"].unique()

array([ 2,  6,  8, 11, 12, 13,  7, 10,  9])

In [7]:
#Drop rows with shopid equals to 111111111
order.drop(order[order['shopid'] == 111111111].index, inplace=True)

In [8]:
order = order.reset_index()

In [9]:
#Calculate number of order cancelled
temp = 0
for stat in order["status"]:
  if (stat >= 6) and (stat <= 10):
    temp = temp + 1

print("Number of order cancelled:",temp)


Number of order cancelled: 2555


In [10]:
#Calculate value of order cancelled
temp = 0
for i in range(len(order.status)):
  if (order.status[i] >= 6) and (order.status[i] <= 10):
    temp = temp + order.total_price[i]

print("Value of order cancelled:","SGD","{:.2f}".format(temp))

Value of order cancelled: SGD 194768.74


In [11]:
#Calculate percentage of Unpaid and Cancelled
temp = 0
total = 0
for i in range(len(order.status)):
  total = total + 1
  if (order.status[i] == 6):
    temp = temp + 1

percentage = (temp / total)*100

print("percentage of Unpaid and Cancelled : ","{:.2f}".format(percentage),"%")

percentage of Unpaid and Cancelled :  21.53 %


In [12]:
#Calculate percentage of Paid then Cancelled 
temp = 0
total = 0
for i in range(len(order.status)):
  total = total + 1
  if (order.status[i] >= 7) and (order.status[i] <= 10):
    temp = temp + 1

percentage = (temp / total)*100

print("percentage of Paid then Cancelled : ","{:.2f}".format(percentage),"%")

percentage of Paid then Cancelled :  10.77 %


In [13]:
#Merge two sheets in excel file
all = order.merge(payment, on='checkoutid', how='inner')

In [14]:
all.head()

,index,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y
0,2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00
1,4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT
2,13,5791188797,373007278,100223545,1419307747,21.155060,SGD,11,28/10/2019 08:27,2019-10-30 01:21:00,NaT,21.155060,SGD,Credit Card,2019-10-30 01:21:00
3,14,7926278218,299238747,100223545,1434961033,130.597117,SGD,11,14/11/2019 12:55,2019-11-14 12:57:00,NaT,130.597117,SGD,Credit Card,2019-11-14 12:57:00
4,16,6802357712,862015688,100223545,1583161237,70.865294,SGD,12,12/11/2019 00:30,2019-11-12 19:33:00,NaT,70.865294,SGD,Credit Card,2019-11-12 19:33:00


In [15]:
all.groupby(["payment_method"]).sum()

,index,orderid,shopid,userid,checkoutid,total_price_x,status,total_price_y
payment_method,,,,,,,,
Bank Transfer,4586039,6157454343967,630990822328,632245208484,1717835910109,88053.291730,9873,88617.411688
Credit Card,25114492,34402210229456,3434720274015,3462917627602,9382920552384,477002.010306,54267,477565.288622
ShopeePay,1562263,2047865384020,213081358708,214276599222,577843970588,29283.567568,3400,29283.567568


In [16]:
#Calculate the cancellation rate
temp = 0
temp2 = 0
temp3 = 0
total = 0
total2 = 0
total3 = 0
for i in range(len(all.payment_method)):
  if (all.payment_method[i] == "Bank Transfer"):
    total = total + 1
    if (order.status[i] >= 6) and (order.status[i] <= 10):
      temp = temp + 1
  elif (all.payment_method[i] == "Credit Card"):
    total2 = total2 + 1
    if (order.status[i] >= 6) and (order.status[i] <= 10):
      temp2 = temp2 + 1
  elif (all.payment_method[i] == "ShopeePay"):
    total3 = total3 + 1
    if (order.status[i] >= 6) and (order.status[i] <= 10):
      temp3 = temp3 + 1

percentage = (temp / total)*100
percentage2 = (temp2 / total2)*100
percentage3 = (temp3 / total3)*100

print("Bank transfer:","{:.2f}".format(percentage),"%")
print("Credit card:","{:.2f}".format(percentage2),"%")
print("ShopeePay:","{:.2f}".format(percentage3),"%")

Bank transfer: 31.62 %
Credit card: 32.46 %
ShopeePay: 31.93 %


In [17]:
#Merge data for reorder analysis
reorder = order.merge(payment, on='checkoutid', how='left')

In [18]:
reorder.head()

,index,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y
0,0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,NaN,NaN,NaT
1,1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,NaN,NaN,NaT
2,2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00
3,3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,NaN,NaN,NaT
4,4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT


In [19]:
#Calculate the time difference of payment and cancel time
reorder["duration"] = ""
for i in range(len(reorder.orderid)):
  temp =  reorder.payment_time_y[i] - reorder.cancel_time[i] 
  reorder["duration"][i] = temp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
reorder

,index,orderid,shopid,userid,checkoutid,total_price_x,currency_x,status,created_time,payment_time_x,cancel_time,total_price_y,currency_y,payment_method,payment_time_y,duration
0,0,6505222765,900457733,100223545,1044269436,147.168235,SGD,2,14/11/2019 21:33,2019-11-16 12:54:00,NaT,NaN,NaN,NaN,NaT,NaT
1,1,6135926959,586311226,100223545,1211270092,33.169735,SGD,2,13/12/2019 19:02,2019-12-13 21:33:00,NaT,NaN,NaN,NaN,NaT,NaT
2,2,5513066971,949679556,100223545,1676514901,24.112195,SGD,2,31/12/2019 17:51,2019-12-31 17:55:00,NaT,24.112195,SGD,Credit Card,2019-12-31 17:55:00,NaT
3,3,5123701563,894942828,100223545,1211270092,75.913637,SGD,6,13/12/2019 19:02,NaT,2019-12-15 21:38:00,NaN,NaN,NaN,NaT,NaT
4,4,7311653202,389950494,100223545,1880489412,14.455363,SGD,6,22/12/2019 14:47,NaT,2019-12-25 11:44:00,14.455363,SGD,Credit Card,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7906,7981,2202494251,426865777,998474908,1948614779,12.589844,SGD,7,29/11/2019 10:14,2019-11-29 10:40:00,2019-12-01 06:41:00,12.589844,SGD,Bank Transfer,2019-11-29 10:40:00,-2 days +03:59:00
7907,7983,3163438450,404024686,998474908,1948614779,2.942826,SGD,10,29/11/2019 10:14,2019-11-29 10:40:00,2019-11-30 00:49:00,12.589844,SGD,Bank Transfer,2019-11-29 10:40:00,-1 days +09:51:00
7908,7984,1041945614,155524980,998474908,1948614779,74.079080,SGD,11,29/11/2019 10:14,2019-11-29 10:40:00,NaT,12.589844,SGD,Bank Transfer,2019-11-29 10:40:00,NaT
7909,7985,2976193751,591090505,998474908,1948614779,64.700863,SGD,11,29/11/2019 10:14,2019-11-29 10:40:00,NaT,12.589844,SGD,Bank Transfer,2019-11-29 10:40:00,NaT


In [21]:
#Get all the difference with positive value which means there is a reorder of the customer
x = 0
for i in reorder.duration:
  if i.days >= 0 and i.seconds > 0:
    x = x + 1
    print("Duration is :",i.days,"days","and",i.seconds,"seconds")
print("Total number of reorder :",x)

Duration is : 29 days and 64740 seconds
Total number of reorder : 1
